In [1]:
import os
import sys
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim
import torchvision.utils as vutils

from IPython.display import HTML
import torchvision.datasets as dset

import tensorflow as tf

import models
import models.research
import models.research.slim
import models.research.slim.nets as nets
from cleverhans.model_zoo.resNet import ResNet
from cleverhans.model_zoo.alexNet import AlexNet

from torch.utils.data.sampler import SubsetRandomSampler


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
from cleverhans.dataset import MNIST

from cleverhans.dataset import CIFAR10


tf.reset_default_graph()

tf.set_random_seed(1234)
sess = tf.Session()

train_start = 0
train_end = 1000
test_start = 1000
test_end = 1200

mnist = MNIST(train_start=train_start, train_end=train_end,
                test_start=test_start, test_end=test_end)


# mnist = CIFAR10(train_start=train_start, train_end=train_end,
#                 test_start=test_start, test_end=test_end)
x_train, y_train = mnist.get_set('train')
x_test, y_test = mnist.get_set('test')


x_train = tf.image.resize(images=x_train, size=(224,224))
x_train = tf.image.grayscale_to_rgb(x_train)


x_test = tf.image.resize(images=x_test, size=(224,224))
x_test = tf.image.grayscale_to_rgb(x_test)


x_train = sess.run(x_train)
x_test = sess.run(x_test)

img_rows, img_cols, nchannels = x_train.shape[1:4]
print(img_rows)
print(img_cols)
print(nchannels)




224
224
3


In [3]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4)

x = tf.placeholder(tf.float32, shape=(None, 224, 224, 3))
y = tf.placeholder(tf.float32, shape=(None, 2))

nb_classes = 2 
nb_filters = 64 
model = AlexNet(scope = "model1", nb_classes = nb_classes, nb_filters = nb_filters)
logits = model.get_logits(x)

loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)
train_op = optimizer.minimize(loss)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)


def get_one_hot(y):
    data_num = y.shape[0]
    
    odd_Array = np.array([0,1,0,1,0,1,0,1,0,1])
    odd_Matrix = np.repeat(odd_Array[ np.newaxis, :], data_num, axis=0)
    
    y_mul = odd_Matrix*y
    odd = np.sum(y_mul, axis=1).astype(int)
    
    y_one_hot = np.zeros((data_num, 2))
        
    y_one_hot[np.arange(data_num), odd] = 1
    return y_one_hot, odd
    


hello
(?, 224, 224, 3)


Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

(1000, 224, 224, 3)
(200, 224, 224, 3)
(1000, 10)
(200, 10)


In [4]:

y_train, _ = get_one_hot(y_train)
y_test, _ = get_one_hot(y_test)

print(y_train.shape)
print(y_test.shape)

init_op = tf.global_variables_initializer()
sess.run(init_op)
dataset_size=x_train.shape[0]

(1000, 2)
(200, 2)


In [5]:
epoch = 1
batch_size = 64
iter_num = dataset_size//batch_size


print(x_train.shape)
for n in range(epoch):
    
    print(n)
    indices = list(range(dataset_size))
    random_seed = n
    np.random.seed(random_seed)
    np.random.shuffle(indices)
    for i in range(iter_num):
        select = indices[i*batch_size:i*batch_size+batch_size]
        x_batch = x_train[select]
        y_batch = y_train[select]
        sess.run(train_op, feed_dict={x: x_batch, y: y_batch})


        
        

(1000, 224, 224, 3)
0


In [6]:
# saver = tf.train.Saver(max_to_keep=10, keep_checkpoint_every_n_hours=0.5)
# saver.save(sess, 'res_50_model.ckpt')

In [7]:

success = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
# Calculate accuracy
accuracy = tf.reduce_mean(tf.cast(success, "float"))


In [8]:
print(sess.run(accuracy, feed_dict={x: x_test, y: y_test}))

0.63


In [9]:
from cleverhans.train import train
from cleverhans.utils_tf import model_eval
from cleverhans.attacks import CarliniWagnerL2
from cleverhans.model_zoo.alexNet import AlexNet
import os

In [11]:
VIZ_ENABLED = True
BATCH_SIZE = 128
NB_EPOCHS = 6
SOURCE_SAMPLES = 100
LEARNING_RATE = .0001
CW_LEARNING_RATE = .2
ATTACK_ITERATIONS = 400
MODEL_PATH = os.path.join('models', 'mnist')
TARGETED = True
source_samples = SOURCE_SAMPLES
attack_iterations = ATTACK_ITERATIONS

In [12]:
train_params = {
  'nb_epochs': 1,
  'batch_size': 128,
  'learning_rate': 0.0001,
  'filename': os.path.split(MODEL_PATH)[-1]
}
rng = np.random.RandomState([2017, 8, 30])

In [13]:

cw = CarliniWagnerL2(model, sess=sess)
saver = tf.train.Saver()
saver.save(sess, MODEL_PATH)

/Users/zhekunluo/Desktop/294-82/Engineering-and-Debugging-Tools-for-Deep-Neural-Networks-/cleverhans/attacks_tf.py:27: UserWarning: attacks_tf is deprecated and will be removed on 2019-07-18 or after. Code should import functions from their new locations directly.
  warnings.warn("attacks_tf is deprecated and will be removed on 2019-07-18"


'models/mnist'

In [14]:
adv_inputs = np.array([[instance] * nb_classes for instance in x_test[:source_samples]], dtype=np.float32)
adv_inputs = adv_inputs.reshape( (source_samples * nb_classes, img_rows, img_cols, nchannels))

one_hot = np.zeros((nb_classes, nb_classes))
one_hot[np.arange(nb_classes), np.arange(nb_classes)] = 1
adv_ys = np.array([one_hot] * source_samples, dtype=np.float32).reshape((source_samples * nb_classes, nb_classes))
yname = "y_target"

cw_params_batch_size = source_samples * nb_classes
cw_params = {'binary_search_steps': 1,
             yname: adv_ys,
             'max_iterations': attack_iterations,
             'learning_rate': CW_LEARNING_RATE,
             'batch_size': cw_params_batch_size,
             'initial_const': 10}

In [15]:
adv = cw.generate_np(adv_inputs, **cw_params)

x_clean = np.array( [[instance] * nb_classes for instance in x_test[:source_samples]], dtype=np.float32)
x_clean = x_clean.reshape((source_samples * nb_classes, img_rows, img_cols, nchannels))



[INFO 2019-12-11 20:22:17,635 cleverhans] Constructing new graph for attack CarliniWagnerL2



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


/Users/zhekunluo/Desktop/294-82/Engineering-and-Debugging-Tools-for-Deep-Neural-Networks-/cleverhans/compat.py:22: UserWarning: <function reduce_sum_v1 at 0x144ae8a60> is deprecated. Switch to calling the equivalent function in tensorflow.  This function was originally needed as a compatibility layer for old versions of tensorflow,  but support for those versions has now been dropped.
  warnings.warn(str(f) + " is deprecated. Switch to calling the equivalent function in tensorflow. "
/Users/zhekunluo/Desktop/294-82/Engineering-and-Debugging-Tools-for-Deep-Neural-Networks-/cleverhans/compat.py:22: UserWarning: <function reduce_max_v1 at 0x144af0950> is deprecated. Switch to calling the equivalent function in tensorflow.  This function was originally needed as a compatibility layer for old versions of tensorflow,  but support for those versions has now been dropped.
  warnings.warn(str(f) + " is deprecated. Switch to calling the equivalent function in tensorflow. "


In [18]:
diff_reshape = np.abs(adv-x_clean).reshape(-1, img_rows*img_cols*nchannels)
diff_sum = np.sum(diff_reshape, axis=1)
avg_perturbation = np.mean(diff_sum)
print(avg_perturbation)

(200,)
0.03390149
